In [1]:
import numpy as np
import cv2

imsize = 400

# Load and resize the content image
content = cv2.imread("./images/house 2-small.jpg")
content = cv2.cvtColor(content, cv2.COLOR_BGR2RGB)
content = content[0:imsize, 0:imsize]

# Load and resize the style image
style = cv2.imread("./images/starry-night - small.jpg")
style = cv2.cvtColor(style, cv2.COLOR_BGR2RGB)
style = style[0:imsize, 0:imsize]


In [3]:
# Define a function for style transfer (you need to implement this function)
def style_transfer(content, style,hallucinated , mask, hallcoeff, wcoeff, patch_sizes, scales, imsize):
    # Initialize variables
    C0 = content.reshape(-1)
    S0 = style.reshape(-1)

    # One-dimensional image
    sigma_s = 5
    sigma_r = 0.2
    h0 = imsize
    w0 = imsize
    c = 3

    # Initialize C to color palette of S using histogram matching
    C0 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply((content * 255).astype(np.uint8))
    C0 = C0 / 255  # Rescale back to the range [0, 1]
    C0 = C0.reshape(-1)

    gap_sizes = [28, 18, 9, 6]
    X = C0
    X = X + np.max(X) * np.random.randn(X.size)  # Add large noise at the beginning
    X = X.reshape(-1)
    
    return hallucinated